In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:99% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:22pt;}
div.text_cell_render.rendered_html{font-size:18pt;}
div.text_cell_render ul li pre{font-size:22pt; line-height:30px;}
div.output {font-size:22pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:22pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:22pt;padding:5px;}
table.dataframe{font-size:22px;}
</style>
"""))

<b><font size="7" color="red">ch14. 웹데이터 수집 - 동적</font></b>
# 1절. Selenium을 이용한 동적 웹크롤링 문법
- Selenium docs : https://selenium-python.readthedocs.io/
`pip install selenium`
- import에서 에러날 경우
`pip install requests`나 `conda install urllib3==1.26.18`

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

In [10]:
driver = webdriver.Chrome()
driver.get("http://www.python.org")
elem = driver.find_element(By.NAME, 'q')
elem.clear()
elem.send_keys('pycon')

In [3]:
from selenium.webdriver.common.keys import Keys
elem.send_keys(Keys.RETURN) # Enter를 입력

In [4]:
btn_elem = driver.find_element(By.CSS_SELECTOR, 'button#submit')
btn_elem.click()

In [ ]:
result_list = driver.find_elements(By.CSS_SELECTOR, 'li > h3 > a')
for result in result_list:
    title = result.text
    link  = result.get_attribute('href')
    print('{} - {}'.format(title, link))

In [6]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(driver.page_source, 'html.parser')
result_list = soup.select('li > h3 > a')
for result in result_list:
    print("{} - {}".format(result.text, result.attrs['href']))

PSF PyCon Trademark Usage Policy - /psf/trademarks/pycon
PyCon Italia 2016 (PyCon Sette) - /events/python-events/378/
PyCon Australia 2013 - /events/python-events/57/
PyCon AU 2019 - /events/python-events/776/
PyCon NL 2025 - /events/python-events/2084/
PyCon Australia 2014 - /events/python-events/10/
PyCon Ireland 2012 - /events/python-events/76/
PyCon Ireland 2016 - /events/python-events/429/
PyCon Ireland 2022 - /events/python-events/1320/
PyCon Australia 2014 - /events/python-events/1447/
PyCon Ireland 2024 - /events/python-events/1862/
PyCon APAC 2025 - /events/python-events/1879/
PyCon AU 2018 - /events/python-events/696/
PyCon APAC 2022 - /events/python-events/1216/
PyCon PH 2024 - /events/python-events/1661/
PyCon Ireland 2023 - /events/python-events/1568/
PyCon PL 2014 - /events/python-events/191/
PyCon MY 2015 - /events/python-events/313/
PyCon Ireland 2015 - /events/python-events/333/
PyCon AU 2015 - /events/python-events/273/


In [7]:
from urllib.parse import urlparse
current_url =driver.current_url # 셀레니윰을 통해 접근한 url
print(current_url)
parse_url = urlparse(current_url)
print(parse_url)
domain = f'{parse_url.scheme}://{parse_url.netloc}'
print(domain)

https://www.python.org/search/?q=pycon&submit=
ParseResult(scheme='https', netloc='www.python.org', path='/search/', params='', query='q=pycon&submit=', fragment='')
https://www.python.org


In [8]:
soup = BeautifulSoup(driver.page_source, 'html.parser')
result_list = soup.select('li > h3 > a')
for result in result_list:
    print("{} - {}".format(result.text, domain+result.attrs['href']))

PSF PyCon Trademark Usage Policy - https://www.python.org/psf/trademarks/pycon
PyCon Italia 2016 (PyCon Sette) - https://www.python.org/events/python-events/378/
PyCon Australia 2013 - https://www.python.org/events/python-events/57/
PyCon AU 2019 - https://www.python.org/events/python-events/776/
PyCon NL 2025 - https://www.python.org/events/python-events/2084/
PyCon Australia 2014 - https://www.python.org/events/python-events/10/
PyCon Ireland 2012 - https://www.python.org/events/python-events/76/
PyCon Ireland 2016 - https://www.python.org/events/python-events/429/
PyCon Ireland 2022 - https://www.python.org/events/python-events/1320/
PyCon Australia 2014 - https://www.python.org/events/python-events/1447/
PyCon Ireland 2024 - https://www.python.org/events/python-events/1862/
PyCon APAC 2025 - https://www.python.org/events/python-events/1879/
PyCon AU 2018 - https://www.python.org/events/python-events/696/
PyCon APAC 2022 - https://www.python.org/events/python-events/1216/
PyCon PH 2

In [11]:
driver.close() # 브라우저 종료

# 2절. 동적웹크롤링 예제
## 2.1 다음뉴스검색

In [13]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
driver = webdriver.Chrome()
driver.get('https://www.daum.net/')
# driver.implicitly_wait(0.5) # 최대로 0.5초동안 대기
time.sleep(0.5) # 0.5초 멈춤
driver.find_element(By.CLASS_NAME, 'tf_keyword').click()
query = input('검색할 단어는?')
driver.find_element(By.CSS_SELECTOR, 'input[type="text"]').send_keys(query)
driver.find_element(By.CLASS_NAME, 'btn_ksearch').click()

검색할 단어는?비트코인


In [15]:
# 뉴스 탭 클릭
# driver.find_elements(By.CSS_SELECTOR, 'ul.list_tab > li')[1].click()
driver.find_element(By.LINK_TEXT, '뉴스').click()

In [20]:
# 뉴스 제목과 link를 list 추가
news_list = []
strongs = driver.find_elements(By.CSS_SELECTOR, 'div.item-title > strong')
# print(len(strongs))
for strong in strongs:
    a = strong.find_element(By.TAG_NAME, 'a')
    title = a.text
    link = a.get_attribute('href')
    news_list.append([title, link])    

In [24]:
# 다음페이지로
page_nav = driver.find_element(By.CSS_SELECTOR, 'div.inner_paging')
print(page_nav.text)
next_page = page_nav.find_element(By.LINK_TEXT, "2")
next_page.click()

1 2 3 4 5


In [25]:
driver.close()

## 2.2 페이징 처리
- 다음 뉴스 페이징 처리 : 원하는 keyword를 원하는 페이지만큼 추출

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
driver = webdriver.Chrome()
driver.get('https://www.daum.net/')
# driver.implicitly_wait(0.5) # 최대로 0.5초동안 대기
time.sleep(0.5) # 0.5초 멈춤
driver.find_element(By.CLASS_NAME, 'tf_keyword').click()
query = input('검색할 단어는?')
driver.find_element(By.CSS_SELECTOR, 'input[type="text"]').send_keys(query)
driver.find_element(By.CLASS_NAME, 'btn_ksearch').click()